# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [12]:
import requests
import pandas as pd

## Получаем cписок из limit_ объектов

In [13]:
offset_ = 1
limit_ = 10
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()

In [14]:
objects_data.keys()

dict_keys(['data', 'errcode'])

In [15]:
objects_data.get('data').keys()

dict_keys(['list', 'total'])

In [16]:
 objects_data.get('data').get('list')

[{'objId': 13383,
  'developer': {'devId': 306,
   'shortName': '2МЕН ГРУПП',
   'fullName': '2МЕН ГРУПП',
   'groupName': '2МЕН ГРУПП ДЕВЕЛОПМЕНТ',
   'orgForm': {'id': 1, 'fullForm': 'Акционерное общество', 'shortForm': 'АО'},
   'companyGroup': 699421001,
   'devInn': '7701651356'},
  'rpdRegionCd': 72,
  'objAddr': 'г Тюмень',
  'objFloorMin': 4,
  'objFloorMax': 20,
  'objElemLivingCnt': 68,
  'objReady100PercDt': '2022-06-30',
  'objSquareLiving': 6289.98,
  'objCompensFundFlg': 1,
  'objProblemFlg': 0,
  'latitude': 57.153,
  'longitude': 65.5343,
  'problemFlag': 'NONE',
  'siteStatus': 'Строящиеся',
  'objGuarantyEscrowFlg': 0,
  'residentialBuildings': 1,
  'rpdNum': '72-000095',
  'pdId': 28105,
  'buildType': 'Жилое',
  'objStatus': 0},
 {'objId': 13397,
  'developer': {'devId': 306,
   'shortName': '2МЕН ГРУПП',
   'fullName': '2МЕН ГРУПП',
   'groupName': '2МЕН ГРУПП ДЕВЕЛОПМЕНТ',
   'orgForm': {'id': 1, 'fullForm': 'Акционерное общество', 'shortForm': 'АО'},
   'companyG

In [17]:
objects_list = objects_data.get('data').get('list')

In [18]:
objids = [x.get('objId') for x in objects_list]

In [19]:
len(objids)

10

In [20]:
objids[:10]

[13383, 13397, 13398, 13399, 23682, 36321, 36912, 44076, 23494, 27476]

## Получаем нормализованные данные, cохраняем в pandas dataframe

In [21]:
import json
from io import StringIO

# json_string = json.dumps(objects_list)
# with open('my_json_data.json', 'w') as file:
#    json.dump(json_string, file)
#df = pd.read_json(StringIO(json.dumps(objects_list)))

df = pd.json_normalize(objects_list)


In [22]:
df.to_excel("out_excel.xlsx", sheet_name='sheet1')

## Cохраняем pandas dataframe в формате pickle

In [23]:
df.to_pickle("out_pickle.pkl")

## Cохраняем pandas dataframe в БД

In [24]:
import sqlite3

conn = sqlite3.connect('test_database1.sqlite')
df.to_sql('builded_objects', conn, if_exists='replace', index=False)
conn.commit()
conn.close()

## Данные по определенному объекту

In [25]:
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/36774'
res = requests.get(url)
res.json()

{'data': {'id': 36774,
  'pdId': 351714,
  'developer': {'devId': 6926,
   'devShortCleanNm': 'ОКЕАН',
   'devShortNm': 'ОКЕАН',
   'devFullCleanNm': 'ОКЕАН',
   'problObjCnt': 0,
   'buildObjCnt': 1,
   'comissObjCnt': 0,
   'regRegionDesc': 'Башкортостан Республика',
   'devPhoneNum': '+7(927)300-31-01',
   'devSite': 'altun-yar.ru',
   'devEmail': 'khabibovr@mail.ru',
   'devInn': '0277093608',
   'devOgrn': '1080277004521',
   'devKpp': '027701001',
   'devLegalAddr': 'Башкортостан Республика, город Уфа, проспект Проспект Октября дом 85',
   'devFactAddr': 'Респ Башкортостан Город Уфа Проспект Октября д. 85',
   'lastRpdId': 511041,
   'fundGuarantyFlg': 0,
   'devOrgRegRegionCd': 2,
   'devEmplMainFullNm': 'Хабибов Рамиль Хамидуллович',
   'developerGroupName': 'Океан',
   'orgForm': {'id': 4,
    'fullForm': 'Общество с ограниченной ответственностью',
    'shortForm': 'ООО'},
   'companyGroupId': 5767476001,
   'objGuarantyEscrowFlg': 1,
   'govFundFlg': 0},
  'region': 2,
  'add

## Cкрипт, который скачивает фотографии всех строящихся объектов с сайта “наш.дом.рф”

In [26]:
url_pattern = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{}'
path = 'builded_objects/'

import pathlib

pathlib.Path(path).mkdir(parents=True, exist_ok=True)

for elem in objects_list:
    objId = elem.get('objId')
    object_info_url = url_pattern.format(objId)
    json = requests.get(object_info_url).json().get('data')
    dir_path = f'{path}/{objId}'
    pathlib.Path(dir_path).mkdir(parents=True, exist_ok=True)
    listOfUrlsGen = (el.get('objRenderPhotoUrl') for el in json.get('photoRenderDTO'))
    for picture_num, picture_url in enumerate(listOfUrlsGen):
        with open(f'{dir_path}/{picture_num}.jpg', 'wb') as handle:
            response = requests.get(picture_url, stream=True)
            handle.write(response.content)
